In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
import datetime

# Load data
def load_data():
    url = "BTC_99.csv"  # replace with the path to your data
    data = pd.read_csv(url)
    data['datetime'] = pd.to_datetime(data['date'])
    data = data.loc[data['datetime'] >= pd.to_datetime('2023-01-01', utc=True)]
    data = data[['datetime', 'close']]  # replace 'close' with the name of your close price column
    return data

# Create dataset
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

# Train model
def train_model(data):
    dataset = data.values
    dataset = dataset.astype('float32')
    
    # Normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)
    
    # Split into train and test sets
    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]
    
    # Prepare the dataset
    look_back = 20  # Increased look_back
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    
    # Reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    
    # Create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(4, input_shape=(1, look_back)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, epochs=100, batch_size=256, verbose=2)
    
    # Make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    
    # Invert predictions and targets to unscaled
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    
    # Calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    print('Test Score: %.2f RMSE' % (testScore))
    
    return model, scaler

# Main function to train the model and make predictions
def main():
    data = load_data()
    while True:
        current_time = datetime.datetime.now()
        if current_time.minute == 0:  # Retrain and predict every hour
            model, scaler = train_model(data)
            # Add your prediction and plotting code here...
            data = load_data()  # Update the data

main()


In [2]:
from okx_api import Market
from pprint import pprint

if __name__ == '__main__':
    market = Market()
    result = market.get_history_index_candles(instId='BTC-USD', bar='1D', limit='100')
    if result:
        pprint(result)
    else:
        print("Failed to fetch historical data.")


{'code': '0',
 'data': [['1696780800000', '27910.8', '27996.2', '27395.5', '27451.7', '1'],
          ['1696694400000', '27956.2', '28102.7', '27715.7', '27910.7', '1'],
          ['1696608000000', '27953.7', '28276.7', '27809.8', '27956.1', '1'],
          ['1696521600000', '27561.3', '28052.7', '27191.3', '27953.8', '1'],
          ['1696435200000', '27401.6', '28118.6', '27399.1', '27561.1', '1'],
          ['1696348800000', '27346.4', '27637.6', '27178.9', '27402.1', '1'],
          ['1696262400000', '28056.7', '28134.3', '27251.6', '27346.4', '1'],
          ['1696176000000', '27151.7', '28590.2', '27049.2', '28056.6', '1'],
          ['1696089600000', '27004.4', '27285.5', '26962.6', '27151.7', '1'],
          ['1696003200000', '26889', '27096.1', '26760.9', '27004.3', '1'],
          ['1695916800000', '27012.3', '27302.3', '26690.4', '26889', '1'],
          ['1695830400000', '26290.2', '27037.6', '26107.3', '27011.1', '1'],
          ['1695744000000', '26125.8', '26830.6', '261

In [3]:
result['data']

[['1696780800000', '27910.8', '27996.2', '27395.5', '27451.7', '1'],
 ['1696694400000', '27956.2', '28102.7', '27715.7', '27910.7', '1'],
 ['1696608000000', '27953.7', '28276.7', '27809.8', '27956.1', '1'],
 ['1696521600000', '27561.3', '28052.7', '27191.3', '27953.8', '1'],
 ['1696435200000', '27401.6', '28118.6', '27399.1', '27561.1', '1'],
 ['1696348800000', '27346.4', '27637.6', '27178.9', '27402.1', '1'],
 ['1696262400000', '28056.7', '28134.3', '27251.6', '27346.4', '1'],
 ['1696176000000', '27151.7', '28590.2', '27049.2', '28056.6', '1'],
 ['1696089600000', '27004.4', '27285.5', '26962.6', '27151.7', '1'],
 ['1696003200000', '26889', '27096.1', '26760.9', '27004.3', '1'],
 ['1695916800000', '27012.3', '27302.3', '26690.4', '26889', '1'],
 ['1695830400000', '26290.2', '27037.6', '26107.3', '27011.1', '1'],
 ['1695744000000', '26125.8', '26830.6', '26105.8', '26289.4', '1'],
 ['1695657600000', '26243.6', '26437.9', '26096.9', '26126', '1'],
 ['1695571200000', '26583.2', '26732.8',

In [5]:
import pandas as pd

result

# Extracting the 'data' key from the dictionary
raw_data = result['data']

# Convert the data into a DataFrame
df = pd.DataFrame(raw_data, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
df = df[['Timestamp', 'Open', 'High', 'Low', 'Close']]
# Ensure that the 'Timestamp' column is of integer type before converting to datetime
df['Timestamp'] = df['Timestamp'].astype(int)
# Convert the timestamp to a datetime format (assuming the timestamp is in milliseconds)
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')

print(df.head())  # Display the first few rows of the dataframe


            Timestamp     Open     High      Low    Close
0 2023-09-18 10:00:00  26889.4  27178.8  26889.4  27135.3
1 2023-09-18 09:00:00  26727.8  26936.5  26727.8  26890.6
2 2023-09-18 08:00:00  26718.9  26734.4  26695.5  26728.2
3 2023-09-18 07:00:00  26642.5  26795.8  26642.5  26718.9
4 2023-09-18 06:00:00  26661.7  26702.8  26642.9  26642.9


In [6]:
display(df)

,Timestamp,Open,High,Low,Close
0,2023-09-18 10:00:00,26889.4,27178.8,26889.4,27135.3
1,2023-09-18 09:00:00,26727.8,26936.5,26727.8,26890.6
2,2023-09-18 08:00:00,26718.9,26734.4,26695.5,26728.2
3,2023-09-18 07:00:00,26642.5,26795.8,26642.5,26718.9
4,2023-09-18 06:00:00,26661.7,26702.8,26642.9,26642.9
...,...,...,...,...,...
95,2023-09-14 11:00:00,26335.3,26483.6,26333.9,26457
96,2023-09-14 10:00:00,26290,26341.1,26287.4,26335.6
97,2023-09-14 09:00:00,26309.7,26309.9,26275.6,26288.2
98,2023-09-14 08:00:00,26347,26360.2,26286.4,26309.7


In [21]:
from datetime import datetime

# Convert the string into a datetime object
dt = datetime.strptime("20230901 0:00", "%Y%m%d %H:%M")

# Convert datetime object to timestamp in milliseconds
timestamp_milliseconds = int(dt.timestamp() * 1000)

print(timestamp_milliseconds)


1693497600000


In [4]:
from okx_api import Market
from pprint import pprint

if __name__ == '__main__':
    market = Market()
    
    # Define the timestamp you want to fetch data before
    # This timestamp should be in milliseconds since that's how timestamps are typically represented in these APIs.
    timestamp_after = 1695033123000
    timestamp_before = 1695033123000 - 1000 * 60 * 60 * 24 * 100  # 100 days before
    result = market.get_history_index_candles(instId='BTC-USD', bar='1H', limit='100', before=timestamp_before,after=timestamp_after)
    
    if result:
        display(result)
    else:
        print("Failed to fetch historical data.")


{'code': '0',
 'msg': '',
 'data': [['1695031200000', '26889.4', '27178.8', '26889.4', '27135.3', '1'],
  ['1695027600000', '26727.8', '26936.5', '26727.8', '26890.6', '1'],
  ['1695024000000', '26718.9', '26734.4', '26695.5', '26728.2', '1'],
  ['1695020400000', '26642.5', '26795.8', '26642.5', '26718.9', '1'],
  ['1695016800000', '26661.7', '26702.8', '26642.9', '26642.9', '1'],
  ['1695013200000', '26631.2', '26682.4', '26620.2', '26661.2', '1'],
  ['1695009600000', '26666.2', '26688.7', '26585.5', '26629.4', '1'],
  ['1695006000000', '26674.1', '26687.7', '26594.2', '26666.5', '1'],
  ['1695002400000', '26473.9', '26747.4', '26470.8', '26674.6', '1'],
  ['1694998800000', '26478', '26494.8', '26454.2', '26474.2', '1'],
  ['1694995200000', '26535.8', '26554.6', '26411.2', '26477.3', '1'],
  ['1694991600000', '26496.7', '26540.3', '26463.9', '26536.6', '1'],
  ['1694988000000', '26496.4', '26497.4', '26453.3', '26496.2', '1'],
  ['1694984400000', '26449.1', '26502.1', '26419.2', '2649

In [ ]:
from okx_api import Market
import pandas as pd
import logging
from tqdm import tqdm
import time

# Set up the logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def convert_to_dataframe(result):
    # Convert the data into a DataFrame
    df = pd.DataFrame(result, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df = df[['Timestamp', 'Open', 'High', 'Low', 'Close']]
    # Ensure that the 'Timestamp' column is of integer type before converting to datetime
    df['Timestamp'] = df['Timestamp'].astype(int)
    # Convert the timestamp to a datetime format (assuming the timestamp is in milliseconds)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
    display(df)
    return df
    
def date_to_timestamp(date_str):
    """Convert a date string to a timestamp."""
    return int(pd.Timestamp(date_str).timestamp() * 1000)

def fetch_data_between_dates(instId, start_date, end_date, bar='1D', limit=100):
    """Fetch historical data between start_date and end_date."""
    
    market = Market()
    all_data = []
    
    # Convert start_date to timestamp
    before_timestamp = date_to_timestamp(end_date)
    
    # Convert end_date to timestamp
    start_timestamp = date_to_timestamp(start_date)
    bar_to_milliseconds = {
        '1D': 86400000,
        '1H': 3600000,
        '1M': 86400000 * 30  # Approximation, assuming 30 days in a month
        # ... add other intervals as needed
    }
    
    interval_milliseconds = bar_to_milliseconds[bar]
    logger.info(f"Fetching data from {start_date} to {end_date}...")
    
    # We'll use a while loop, but we don't know the exact number of iterations.
    # Let's use a dummy tqdm loop to update the progress bar manually.
    with tqdm(total=100, desc="Fetching data", dynamic_ncols=True) as pbar:
        previous_length = 0
        consecutive_failures = 0
        MAX_CONSECUTIVE_FAILURES = 5  # You can adjust this as needed
        while True:
            # Fetch a batch of data
            result = market.get_history_index_candles(instId=instId, bar=bar, limit=str(limit), before=str(before_timestamp))
            
            # If no data fetched, try a previous time interval
            if not result or 'data' not in result or not result['data']:
                consecutive_failures += 1
                if consecutive_failures >= MAX_CONSECUTIVE_FAILURES:
                    print("Reached maximum number of consecutive failed attempts. Exiting.")
                    break
                before_timestamp -= interval_milliseconds
                print(f"Data fetching failed for timestamp {before_timestamp}. Retrying with previous timestamp.")
                continue

            # Reset consecutive failures count if we successfully fetched data
            consecutive_failures = 0
                
            # Add the fetched data to all_data
            all_data.extend(result['data'])
            
            # Update the before_timestamp with the timestamp of the earliest data point in the fetched batch
            earliest_timestamp = int(result['data'][-1][0])
            
            # If earliest_timestamp is later than start_timestamp, break out of the loop
            if earliest_timestamp <= start_timestamp:
                print("earliest_timestamp smaller than start_timestamp, exit")
                break
            
            before_timestamp = earliest_timestamp
            
            # Update tqdm progress bar based on the new data fetched
            new_data_length = len(all_data) - previous_length
            pbar.update(new_data_length / limit * 100)
            previous_length = len(all_data)
            
            # Sleep for a short duration to avoid hitting rate limits (if any)
            time.sleep(2)
    
    logger.info("Data fetching complete!")
    
    # Filter out the data which is after the start_date
    all_data = [data for data in all_data if int(data[0]) >= start_timestamp]
    
    return all_data


# Example usage:
if __name__ == '__main__':
    data = fetch_data_between_dates('BTC-USD', '2023-01-01', '2023-10-08')
    print(data)
    result = convert_to_dataframe(data)
    result.to_csv("a.csv",index = True)


INFO:__main__:Fetching data from 2023-01-01 to 2023-10-08...
Fetching data:   1%|▎                         | 1.0/100 [00:01<02:41,  1.63s/it]

Data fetching failed for timestamp 1696694400000. Retrying with previous timestamp.


Fetching data:   2%|▌                         | 2.0/100 [00:06<06:15,  3.83s/it]

Data fetching failed for timestamp 1696694400000. Retrying with previous timestamp.


Fetching data:   3%|▊                         | 3.0/100 [00:13<08:31,  5.27s/it]

...

Fetching data:  28%|███████                  | 28.0/100 [02:47<07:11,  6.00s/it]

Data fetching failed for timestamp 1696694400000. Retrying with previous timestamp.


Fetching data:  29%|███████▏                 | 29.0/100 [02:53<06:58,  5.89s/it]

Data fetching failed for timestamp 1696694400000. Retrying with previous timestamp.


Fetching data:  30%|███████▌                 | 30.0/100 [03:01<07:41,  6.60s/it]